<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_de_corte_de_estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema de corte de estoque

Neste notebook, implementamos um modelo de programação linear para o clássico problema de corte de estoque unidimensional, dado a seguir:

\begin{align}
\min \quad z & = x_1+x_2+x_3+x_3+x_4+x_5 \\
\text{s.a.} &\\
& x_1+2x_3+x_4 \geq 10\\
& x_1+2x_3+x_4 \geq 12\\
& x_1+x_2 \geq 15\\
& 2x_2+x_4+2x_5 \geq 31\\
& x_3+x_5 \geq 17 \\
& x_1, x_2, x_3, x_4, x_5 \geq 0
\end{align}

## Instalação das bibliotecas necessárias

In [ ]:
!pip install pyomo
!apt-get install coinor-cbc

In [ ]:
import pyomo.environ as pyEnv
import numpy as np

## Declaração dos dados do problema

Note que cada padrão é uma coluna da matriz ```padroes```:



In [ ]:
padroes = np.array([[1, 0, 0, 3, 1],
                    [1, 0, 2, 1, 0],
                    [1, 1, 0, 0, 0],
                    [0, 2, 0, 1, 2],
                    [0, 0, 1, 0, 1]])

demandas = np.array([10, 12, 15, 31, 17])

## Declaração do modelo

In [ ]:
m = padroes.shape[0]   ##Número de itens diferentes
n = padroes.shape[1]   ##Número de padroes é o número de colunas

##Criação da instância do modelo
modelo = pyEnv.ConcreteModel()

##Criação dos índices
modelo.Indices = range(n)
modelo.Indices2 = range(m)

##Variáveis de decisão
modelo.x = pyEnv.Var(modelo.Indices, within = pyEnv.NonNegativeReals)    ##Note a declaração das variáveis como não negativas

##Função-objetivo
modelo.Objetivo = pyEnv.Objective(expr = sum(modelo.x[i] for i in modelo.Indices),
                                  sense = pyEnv.minimize)    ##Note a direção de minimização

##Note aqui a declaração das restrições. Para não declarar cada restrição individualmente
##definimos uma função que gera as restrições.

def restricao(modelo, i):
    return sum(padroes[i][j]*modelo.x[j] for j in modelo.Indices) >= demandas[i]

##Note que para cada valor em modelo.Indices2 será criada uma restrição,
##em que o índice i na função restricao assumirá um valor em modelo.Indices2
modelo.rest = pyEnv.Constraint(modelo.Indices2, rule = restricao)


##Solução do modelo

In [ ]:
solver = pyEnv.SolverFactory('cbc')
result_objetivo = solver.solve(modelo, tee=True)

## Print do resultado

In [ ]:
lista = list(modelo.x.keys())
print('\nVariaveis: ')
for i in lista:
    print('x_'+str(i+1), '---', modelo.x[i]())

print('\nValor da função objetivo =', modelo.Objetivo())
